In [0]:
# %cd data
# !pwd
# !gunzip GoogleNews-vectors-negative300.bin
# %cd ..

In [0]:
#INPUTS

#repoPath should start with the path 
#'/content/gdrive/My Drive/' and go to the folder where the git repo is, and should end with the name of the repo:
repoPath = '/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/findingKeyphraseUsingTextRank'
userEmail = 'sanmesh1@gmail.com'
userName = 'sanmesh1'

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
! pip install ipdb -q
import ipdb 

#place the line of code below from whichever line you want to start debugging
#type n for step over and s for step into
# ipdb.set_trace()

In [0]:
import os
os.chdir(repoPath)
#%cd gdrive/My Drive/project_folder/TextSimilarityUsingWord2Vec

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')
content = '''
The Wandering Earth, described as China’s first big-budget science fiction thriller, quietly made it onto screens at AMC theaters in North America this weekend, and it shows a new side of Chinese filmmaking — one focused toward futuristic spectacles rather than China’s traditionally grand, massive historical epics. At the same time, The Wandering Earth feels like a throwback to a few familiar eras of American filmmaking. While the film’s cast, setting, and tone are all Chinese, longtime science fiction fans are going to see a lot on the screen that reminds them of other movies, for better or worse.
'''
doc = nlp(content)
for sents in doc.sents:
    print(sents.text)
    print()


The Wandering Earth, described as China’s first big-budget science fiction thriller, quietly made it onto screens at AMC theaters in North America this weekend, and it shows a new side of Chinese filmmaking — one focused toward futuristic spectacles rather than China’s traditionally grand, massive historical epics.

At the same time, The Wandering Earth feels like a throwback to a few familiar eras of American filmmaking.

While the film’s cast, setting, and tone are all Chinese, longtime science fiction fans are going to see a lot on the screen that reminds them of other movies, for better or worse.




In [10]:
candidate_pos = ['NOUN', 'PROPN', 'VERB']
sentences = []

for sent in doc.sents:
  selected_words = []
  for token in sent:
    if token.pos_ in candidate_pos and token.is_stop is False:
      selected_words.append(token)
  sentences.append(selected_words)
print(sentences)

[[Wandering, Earth, described, China, budget, science, fiction, thriller, screens, AMC, theaters, North, America, weekend, shows, filmmaking, focused, spectacles, China, epics], [time, Wandering, Earth, feels, throwback, eras, filmmaking], [film, cast, setting, tone, Chinese, science, fiction, fans, going, lot, screen, reminds, movies]]


In [0]:

from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [6]:
text = '''
The Wandering Earth, described as China’s first big-budget science fiction thriller, quietly made it onto screens at AMC theaters in North America this weekend, and it shows a new side of Chinese filmmaking — one focused toward futuristic spectacles rather than China’s traditionally grand, massive historical epics. At the same time, The Wandering Earth feels like a throwback to a few familiar eras of American filmmaking. While the film’s cast, setting, and tone are all Chinese, longtime science fiction fans are going to see a lot on the screen that reminds them of other movies, for better or worse.
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(10)

science - 1.7276378287292111
fiction - 1.7123791481736559
filmmaking - 1.4388798751402918
China - 1.4172218029410266
Earth - 1.3088154732297723
Chinese - 1.0971675275482093
fans - 1.0971675275482093
Wandering - 1.0071059904601571
weekend - 1.002449354657688
America - 0.9976329264870932
budget - 0.9769693829073562
North - 0.9711240881032547


In [0]:
from time import time
start_nb = time()

In [0]:
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.

# Remove stopwords.
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
start = time()
import os
from gensim.models import Word2Vec

%cd data/
!ls
import os.path
from os import path
if path.exists("GoogleNews-vectors-negative300.bin") == False:
  !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
  !gunzip GoogleNews-vectors-negative300.bin
else:
  print("datasetAlreadyExists")
%cd ..

from gensim.models.keyedvectors import KeyedVectors
model_path = './data/GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

print('Cell took %.2f seconds to run.' % (time() - start))

/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance/data
GoogleNews-vectors-negative300.bin
datasetAlreadyExists
/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Cell took 71.50 seconds to run.


In [0]:
# Pre-processing a document.

from nltk import word_tokenize
download('punkt')  # Download data for tokenizer.

def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
#Create pandas dataframe on dataset
import pandas as pd
#https://data.world/crowdflower/ecommerce-search-relevance
df = pd.read_csv('https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz', encoding = "ISO-8859-1")

In [0]:
#create source document and target document we will be comparing scores for
numDataPoints = 6000

sourceTitle = df['product_title'][2584]
source_doc = df['product_description'][2584]

targetTitles = df['product_title'][:numDataPoints].tolist()
target_docs = df['product_description'][:numDataPoints].tolist()

In [0]:
#using WmdSimilarity

wmd_corpus = []
documents = []
documentTitles = []

for i in range(len(target_docs)):
    if target_docs[i] == target_docs[i]: #this is to check if there is an empty string
      # ipdb.set_trace()
      target_docs_preproc = preprocess(target_docs[i])
      if target_docs_preproc != []:
        wmd_corpus.append(target_docs_preproc)
      else:
        wmd_corpus.append([])
    else:
      wmd_corpus.append([])
    documentTitles.append(targetTitles[i])
    documents.append(target_docs[i])
# Initialize WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instance = WmdSimilarity(wmd_corpus, model, num_best=10)



In [0]:
start = time()

query = preprocess(source_doc)

sims = instance[query]  # A query is simply a "look-up" in the similarity class.

In [0]:
# Print the query and the retrieved documents, together with their similarities.
print('Query:')
print(sourceTitle)
for i in range(num_best):
    print()
    print('sim = %.4f' % sims[i][1])
    print(documentTitles[sims[i][0]])

Query:
Go Green AAA Alkaline Battery 8-pack

sim = 1.0000
Go Green AAA Alkaline Battery 8-pack

sim = 0.4990
Duracell Procell Alkaline AAA Batteries (Case of 24)

sim = 0.4986
Duracell Coppertop Alkaline AAA Batteries (Case of 24)

sim = 0.4970
Energizer Industrial Alkaline AAA Batteries (Case of 24)

sim = 0.4948
SmartyKat SweetGreens Cat Grass Kit

sim = 0.4865
Innovera Alkaline AA Batteries (Pack of 8)

sim = 0.4859
Rayovac Ultra Pro Alkaline AA Batteries (Case of 48)

sim = 0.4853
Duracell Procell Alkaline AA Batteries (Case of 24)

sim = 0.4840
Medline AAA Alkaline Battery (Case of 144)

sim = 0.4840
Medline MedCell Alkaline Batteries, AA (Case of 144)
